# Check basic properties of SEM (raw) and SEM transaction (generated)

In [1]:
import pandas as pd
sem = pd.read_csv('sem.csv') # raw
sem_trans = pd.read_csv('sem_transactions.csv') # generated

/tmp/user/22018/ipykernel_251650/4061657183.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sem = pd.read_csv('sem.csv')


In [2]:
# we remove the NaN row, as we did in preprocess_sem.py
sem[sem.date.isna()]
print(sem.shape)
sem = sem[~sem.date.isna()]
print(sem.shape)

Series([], Name: amount_sum, dtype: float64)

In [ ]:
# No negative amount_sum values yay!
sem.amount_sum[sem.amount_sum < 0]

In [ ]:
# No negative total_amount values yay!
sem_trans.total_amount[sem_trans.total_amount < 0]

In [4]:
sem_trans.shape

(6990926, 12)


(6990925, 12)


In [6]:
# Load in generated sem_transactions.csv


In [8]:
sem_trans.columns

Index(['time_stamp', 'supplier_t', 'buyer_t', 'hs6', 'bill_count',
       'total_quantity', 'total_amount', 'total_weight'],
      dtype='object')

574552    -3.949134e+06
1032437   -3.058061e+06
1498057   -1.496447e+06
1869520   -2.551070e+03
1889963   -6.008111e+05
2426839   -2.919570e+06
Name: total_amount, dtype: float64

# Check chronological batching

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

# internal imports
from tgb.utils.utils import *
from tgb.linkproppred.evaluate import Evaluator
from modules.emb_module import GraphAttentionEmbedding
from modules.msg_func import TGNPLMessage
from modules.msg_agg import MeanAggregator
from modules.neighbor_loader import LastNeighborLoaderTGNPL
from modules.memory_module import TGNPLMemory, StaticMemory
from modules.early_stopping import  EarlyStopMonitor
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset, PyGLinkPropPredDatasetHyper
from examples.linkproppred.general.tgnpl import *

from scipy.stats import pearsonr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# recreate args
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args = Namespace(dataset='tgbl-hypergraph_sem',
                 lr=1e-4,
                 bs=2000,  # use larger batch size since we're not training
                 k_value=10,
                 num_epoch=100,
                 seed=1,
                 mem_dim=100,
                 time_dim=10,
                 emb_dim=100,
                 tolerance=1e-6,
                 patience=100,
                 num_run=1,
                 wandb=False,
                 bipartite=False,
                 memory_name='static',
                 emb_name='sum',
                 use_inventory=False,
                 debt_penalty=0,
                 consum_rwd=0,
                 gpu=0,
                 num_train_days=-1,
                 use_prev_sampling=False,
                 batch_by_t=False)
device = torch.device("cpu")

In [10]:
data.msg[data.msg<0]

tensor([-3.9491e+06, -3.0581e+06, -1.4964e+06, -2.5511e+03, -6.0081e+05,
        -2.9196e+06])

In [4]:
dataset = PyGLinkPropPredDatasetHyper(name=args.dataset, root="datasets", 
                                      use_prev_sampling = args.use_prev_sampling)
data = dataset.get_TemporalData().to(device)
train_loader, val_loader, test_loader = set_up_data(args, data, dataset)

Dataset tgbl-hypergraph_sem url not found, download not supported yet.
file found, skipping download
Dataset directory is  /lfs/hyperturing1/0/zhiyinl/supply-chains/TGB/tgb/datasets/tgbl_hypergraph_sem
loading processed file


AssertionError: 

In [ ]:
train_loader

In [ ]:
# Check by assertion (timestamp value always increasing): seems correct! 
prev_max_t = 0
for batch in tqdm(train_loader):
    current_min_t, current_max_t = min(set(batch.t)), max(set(batch.t))
    assert(prev_max_t <= current_min_t) # previous max should be <= current minimum
    prev_max_t = current_max_t

In [ ]:
# Check by eye-balling
for batch in tqdm(train_loader):
    print(batch.t)